In [5]:
import json
import sys
import numpy as np
import torch
import torch.utils.data as Data
from torch.utils.data import SubsetRandomSampler, DataLoader
import torch.nn as nn
import torch.optim as optim
from Model import Transformer
import time
import os
from PDB2Angles import extract_backbone_model

In [45]:
def convert_to_Longtensor(data):
    data = torch.LongTensor(data)
    return data

def make_decoderinput(data):
    for i in range(len(data)):
        data[i].insert(0, 1001)
        # data[i].append(36002)
    return data

def recover_target(data):
    for i in range(len(data)):
        del data[i][0]
    return data

def make_decoderoutput(data):
    for i in range(len(data)):
        data[i].append(1002)
    return data

In [398]:
#dataset inputs:
#SS (helix, sheet, coil)
#phi, psi, omega, CCN, CNC, NCC angle
from typing import Any

import pandas as pd


class GetAngleForSSData(Data.Dataset):
    def __init__(self, data_path, ss, angle, batch_size = 4):
        self.data_path = data_path
        self.ss = ss
        self.angle = angle
        self.proteins = os.listdir(data_path)
    def __len__(self):
        return len(self.proteins)
    def __getitem__(self, index) -> Any:
        protein = self.proteins[index]
        print(protein)
        # #enc angle and inflate values for training convenience 
        # train_enc_inputs = convert_to_Longtensor(source_angle)
        
        target = pd.read_csv(os.path.join(self.data_path, protein,"target.csv"), usecols=["SS", self.angle])
        target_angle = target[target["SS"] == self.ss][self.angle].values
        #find the corresponding indices in target.csv that have ss equal to desired self.ss
        target_indices = target[target["SS"] == self.ss].index
        #get the corresponding source angle values
        source = pd.read_csv(os.path.join(self.data_path, protein,"source.csv"), usecols=[ self.angle])
        source_angle = source.iloc[target_indices][self.angle].values
        # train_dec_inputs = convert_to_Longtensor(make_decoderinput(target_angle))
        # train_dec_outputs = convert_to_Longtensor(make_decoderoutput(recover_target(target_angle)))
        
        return source_angle, target_angle#train_dec_inputs, train_dec_outputs

In [399]:
data = GetAngleForSSData("/kuhpc/work/slusky/s300y051/AnglesRefine/example/sample_source_targets", "E", "phi")

In [400]:
# def collate_fn(batch):
#     source_angle = [item[0] for item in batch]
#     target_angle = [item[1] for item in batch]
#     return source_angle, target_angle

In [444]:
def collate_fn(batch):
    train_enc_inputs =  [list(x) for x in list(zip(*batch))[1]]
    #remove any empty arrays
    train_enc_inputs = [x for x in train_enc_inputs if x != []]
    train_enc_inputs = [convert_to_Longtensor(x) for x in train_enc_inputs]
    #remove any -1 values
    target_angles = [list(x) for x in list(zip(*batch))[1]]
    #remove any empty angles
    target_angles = [x for x in target_angles if x != []]
    target_enc_inputs = make_decoderinput(target_angles)
    target_enc_inputs = [convert_to_Longtensor(x) for x in target_enc_inputs]
    #recover then decode for the target_dec
    target_dec_inputs = make_decoderinput(recover_target(target_angles))
    target_dec_inputs = [convert_to_Longtensor(x) for x in target_dec_inputs]
    return train_enc_inputs, target_enc_inputs, target_dec_inputs

In [445]:
train_loader = DataLoader(data,batch_size=5, shuffle=False, collate_fn=collate_fn)
#batch in form:
# [batch_size, tuple (source_angle, target_angle)]

In [449]:
for batch in train_loader:
    train_enc_inputs, target_enc_inputs , target_dec_inputs = batch
    assert len(train_enc_inputs) == len(target_enc_inputs) == len(target_dec_inputs)
    # train_enc_inputs, target_enc_inputs = batch
    # print(len(train_enc_inputs))
    


A0A009IHW8
A0A023I7E1
A0A023YYV9
A0A059TC02
A0A061I403
A0A067CMC7
A0A068J840
A0A072ULZ1
A0A072UR65
A0A084JZF2


4